In [ ]:
import json
import time
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome()  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '연희동 카페'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

# 카페 리스트
cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
cafe_dict = {'카페정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    names = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')  # (3) 장소명
    types = driver.find_elements(By.CSS_SELECTOR, '.KCMnt')  # (4) 장소 유형

    for data in range(len(cafe_list)):  # 카페 리스트 만큼
        print(data)

        sleep(1)
        try:
            # 지번, 도로명 초기화
            jibun_address = ''
            road_address = ''

            # (3) 카페명 가져오기
            cafe_name = names[data].text
            print(cafe_name)

            # (4) 유형
            cafe_type = types[data].text
            print(cafe_type)

            # dict에 데이터 집어넣기
            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type,
                # 'road_address': road_address,
                # 'jibun_address': jibun_address
            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

        except Exception as e:
            print(e)
            print('ERROR!' * 3)

            dict_temp = {
                'name': cafe_name,
                'cafe_type': cafe_type

            }

            cafe_dict['카페정보'].append(dict_temp)
            sleep(1)

    # 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 카페일 경우 다음버튼 클릭
        next_btn[-1].click()

        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.

# 배열에 저장
data = [cafe_dict]
